In [15]:
import biobb_structure_checking
import biobb_structure_checking.constants as cts
from biobb_structure_checking.structure_checking import StructureChecking
base_dir_path=biobb_structure_checking.__path__[0]
args = cts.set_defaults(base_dir_path,{'notebook':True})

## STEP 3

In [16]:
import argparse
import sys
import os
import math

from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.NACCESS import NACCESS_atomic
from Bio.PDB.NeighborSearch import NeighborSearch
from Bio.PDB.PDBIO import PDBIO, Select

**IMPORT PARAMETERS OF THE RESIDUE LIBRARY**

In [17]:
class ResiduesDataLib():
    def __init__(self, fname):
        self.residue_data = {}
        try:
            fh = open(fname, "r")
        except OSError:
            print("#ERROR while loading library file (", fname, ")")
            sys.exit(2)
        for line in fh:
            if line[0] == '#':
                continue
            data = line.split()
            r = Residue(data)
            self.residue_data[r.id] = r
        self.nres = len(self.residue_data)

    def get_params(self, resid, atid):
        atom_id = resid + ':' + atid
        if atom_id in self.residue_data:
            return self.residue_data[atom_id]
        else:
            print("WARNING: atom not found in library (", atom_id, ')')
            return None

Defining classes

In [18]:
class Residue():
    def __init__(self, data):
        self.id = data[0] + ':' + data[1]
        self.at_type = data[2]
        self.charge = float(data[3])
        
class AtomType():
    def __init__(self, data):
        self.id   = data[0]
        self.eps  = float(data[1])
        self.sig  = float(data[2])
        self.mass = float(data[3])
        self.fsrf = float(data[4])
        self.rvdw = self.sig * 0.5612
        
class VdwParamset(): #extracted from GELPI's github
    #parameters for the VdW
    def __init__ (self, file_name):
        self.at_types = {}
        try:
            fh = open(file_name, "r")
        except OSError:
            print ("#ERROR while loading parameter file (", file_name, ")")
            sys.exit(2)
        for line in fh:
            if line[0] == '#':
                continue
            data = line.split()
            self.at_types[data[0]] = AtomType(data)
        self.ntypes = len(self.at_types)
        fh.close()

**LOAD PARAMERTERS FROM THE FILES**

IMPORTANT: Change the pathway for your own

In [19]:
residue_library = ResiduesDataLib('/home/irene/Documents/BIOPYISICS/step 2/residue_library.txt')

ff_params = VdwParamset('/home/irene/Documents/BIOPYISICS/step 2/vdwprm')

In [20]:
# set the pdb_path and load the structure
pdb_path = '/home/irene/Documents/BIOPYISICS/step 2/6m0j_fixed.pdb'
# Setting the Bio.PDB.Parser object
parser = PDBParser(PERMISSIVE=1)
# Loading structure
st = parser.get_structure('st', pdb_path)

**SERCHING FOR ENERGIES TO PUT IN THE FORMULE**

In [21]:
def add_atom_parameters(st, residue_library, ff_params):
    ''' Adds parameters from libraries to atom .xtra field
        For not recognized atoms, issues a warning and put default parameters
    '''
    for at in st.get_atoms():
        resname = at.get_parent().get_resname()
        params = residue_library.get_params(resname, at.id)
        if not params:
            print("WARNING: residue/atom pair not in library ("+atom_id(at) + ')')
            at.xtra['atom_type'] = at.element
            at.xtra['charge'] = 0
        else:
            at.xtra['atom_type'] = params.at_type
            at.xtra['charge'] = params.charge
        at.xtra['vdw'] = ff_params.at_types[at.xtra['atom_type']]

CALL FUNCTION TO ENTER PARAMETERS

- residue_library and ff_ params --> parameters already imported above
- st --> structure of the protein, file imported from the path (above) (load parameters section)

In [22]:
add_atom_parameters(st, residue_library, ff_params)

los atoms que dan error son los que se ubican en la c terminal and n terminal solucion: corregir en el mismo file los warning

- GLY OXT O -0.5722 --> i have to modified from GLYN

In [23]:
def get_interface(st, dist):
    ''' Detects interface residues within a distance(dist)
        Assumes two chains, i.e. a unique interface set per chain.
    '''
    select_ats = []
    for at in st.get_atoms():
        # Skip Hydrogens to reduce time
        if at.element != 'H':
            select_ats.append(at)
    nbsearch = NeighborSearch(select_ats)
    interface = {}
    # Sets are more efficient than lists. Use sets when order is not relevant
    for ch in st[0]:
        interface[ch.id] = set()

    for at1, at2 in nbsearch.search_all(dist):
        #Only different chains
        res1 = at1.get_parent()
        ch1 = res1.get_parent()
        res2 = at2.get_parent()
        ch2 = res2.get_parent()
        if ch1 != ch2:
            interface[ch1.id].add(res1)
            interface[ch2.id].add(res2)
    return interface

In [24]:
get_interface(st, 3.5) # These are the residues of the interface

{'A': {<Residue GLN het=  resseq=24 icode= >,
  <Residue PHE het=  resseq=28 icode= >,
  <Residue ASP het=  resseq=30 icode= >,
  <Residue LYS het=  resseq=31 icode= >,
  <Residue HIE het=  resseq=34 icode= >,
  <Residue GLU het=  resseq=35 icode= >,
  <Residue GLU het=  resseq=37 icode= >,
  <Residue ASP het=  resseq=38 icode= >,
  <Residue TYR het=  resseq=41 icode= >,
  <Residue GLN het=  resseq=42 icode= >,
  <Residue TYR het=  resseq=83 icode= >,
  <Residue LYS het=  resseq=353 icode= >,
  <Residue GLY het=  resseq=354 icode= >,
  <Residue ASP het=  resseq=355 icode= >},
 'E': {<Residue LYS het=  resseq=417 icode= >,
  <Residue GLY het=  resseq=446 icode= >,
  <Residue TYR het=  resseq=449 icode= >,
  <Residue TYR het=  resseq=453 icode= >,
  <Residue ASN het=  resseq=487 icode= >,
  <Residue TYR het=  resseq=489 icode= >,
  <Residue GLN het=  resseq=493 icode= >,
  <Residue GLY het=  resseq=496 icode= >,
  <Residue GLN het=  resseq=498 icode= >,
  <Residue THR het=  resseq=500 ic

**DEFINE ALANINE ATOMS**

In [25]:
ala_atoms = {'N', 'H', 'CA', 'HA', 'C', 'O', 'CB', 'HB', 'HB1', 'HB2', 'HB3', 'HA1', 'HA2', 'HA3'}

In [26]:
def MH_diel(r):
    '''Mehler-Solmajer dielectric'''
    return 86.9525 / (1 - 7.7839 * math.exp(-0.3153 * r)) - 8.5525


def elec_int(at1, at2, r):
    '''Electrostatic interaction energy between two atoms at r distance'''
    return 332.16 * at1.xtra['charge'] * at2.xtra['charge'] / MH_diel(r) / r


def vdw_int(at1, at2, r):
    '''Vdw interaction energy between two atoms'''
    eps12 = math.sqrt(at1.xtra['vdw'].eps * at2.xtra['vdw'].eps)
    sig12_2 = at1.xtra['vdw'].sig * at2.xtra['vdw'].sig
    return 4 * eps12 * (sig12_2**6/r**12 - sig12_2**3/r**6)


def calc_solvation(st, res):
    '''Solvation energy based on ASA'''
    solv = 0.
    solv_ala = 0.
    for at in res.get_atoms():
        if 'EXP_NACCESS' not in at.xtra:
            continue
        s = float(at.xtra['EXP_NACCESS'])* at.xtra['vdw'].fsrf
        solv += s
        if at.id in ala_atoms:
            solv_ala += s
    return solv, solv_ala


def calc_int_energies(st, res):
    '''Returns interaction energies (residue against other chains)
        for all atoms and for Ala atoms
    '''
    elec = 0.
    elec_ala = 0.
    vdw = 0.
    vdw_ala = 0.

    for at1 in res.get_atoms():
        for at2 in st.get_atoms():
        # skip same chain atom pairs
            if at2.get_parent().get_parent() != res.get_parent():
                r = at1 - at2
                e = elec_int(at1, at2, r)
                elec += e
                if at1.id in ala_atoms: #GLY are included implicitly
                    elec_ala += e
                e = vdw_int(at1, at2, r)
                vdw += e
                if at1.id in ala_atoms: #GLY are included implicitly
                    vdw_ala += e
    return elec, elec_ala, vdw, vdw_ala

In [27]:
def residue_id(res):
    '''Returns readable residue id'''
    return '{} {}{}'.format(res.get_resname(), res.get_parent().id, res.id[1])


def atom_id(at):
    '''Returns readable atom id'''
    return '{}.{}'.format(residue_id(at.get_parent()), at.id)

**CALCULATION OF THE TOTAL ENERGY WITH ALA** (for all the residues)

IMPORTANT: 
- Change the NACCESS_BINARY pathway for your own
- Check if the pathway in naccess file (EXE_PATH) is correct

In [28]:
NACCESS_BINARY = '/home/irene/Documents/BIOPYISICS/step 2/Parameters_solvation/soft/NACCESS/naccess'
srf = NACCESS_atomic(st[0], naccess_binary = NACCESS_BINARY)
io = PDBIO()
st_chains = {}

# Using BioIO trick (see tutorial) to select chains
class SelectChain(Select):
    def __init__(self, chid):
        self.id = chid

    def accept_chain(self, chain):
        if chain.id == self.id:
            return 1
        else:
            return 0

for ch in st[0]:
    io.set_structure(st)
    io.save('tmp.pdb', SelectChain(ch.id))
    if ch.id == "A":
        st_chains[ch.id] = parser.get_structure('stA', 'tmp.pdb')
        add_atom_parameters(st_chains[ch.id], residue_library, ff_params)
        srfA = NACCESS_atomic(st_chains[ch.id][0], naccess_binary=NACCESS_BINARY)
    else:
        st_chains[ch.id] = parser.get_structure('stE', 'tmp.pdb')
        add_atom_parameters(st_chains[ch.id], residue_library, ff_params)
        srfE = NACCESS_atomic(st_chains[ch.id][0], naccess_binary=NACCESS_BINARY)

os.remove('tmp.pdb')


# Interface residues
max_dist = 3.5

if max_dist > 0.:
    interface = get_interface(st, max_dist)


## Initialize Energy aggregates
elec = {}
elec_ala = {}

vdw = {}
vdw_ala = {}

solvAB = {}
solvAB_ala = {}

solvA = {}
solvA_ala = {}

totalIntElec = 0.
totalIntVdw = 0.
totalSolv = 0.
totalSolvMon = {}

## Get the chains ids
chids = []
for ch in st[0]:
    chids.append(ch.id)
    totalSolvMon[ch.id] = 0

total = 0.

for ch in st[0]:
    for res in ch.get_residues():
        if max_dist > 0 and res not in interface[chain.id]:
            continue
        elec[res], elec_ala[res], vdw[res], vdw_ala[res] = calc_int_energies(st[0], res)
        solvAB[res], solvAB_ala[res] = calc_solvation(st[0], res)
        solvA[res], solvA_ala[res] = calc_solvation(
            st_chains[ch.id],
            st_chains[ch.id][0][ch.id][res.id[1]]
        )
        totalIntElec += elec[res]
        totalIntVdw += vdw[res]
        totalSolv += solvAB[res]
        totalSolvMon[ch.id] += solvA[res]
        total += elec[res] + vdw[res] + solvAB[res] - solvA[res]

print()
print("Energies when residues in the interface are mutated to alanine")
print('Total Electrostatic Int. =', totalIntElec)
print('Total Vdw Int. =', totalIntVdw)
print('Total Solvation Complex =', totalSolv)
print('Total Solvation', chids[0], '=', totalSolvMon[chids[0]])
print('Total Solvation', chids[1], '=', totalSolvMon[chids[1]])
print('DGint =', total)

/home/irene/anaconda3/lib/python3.9/site-packages/Bio/PDB/NACCESS.py:77: UserWarning: set: Variable name must begin with a letter.

  warnings.warn(err)


Exception: NACCESS did not execute or finish properly.

### Process only if chain is A or E

**energies for each residue in Chain A or E**

In [30]:
NACCESS_BINARY = '/home/irene/Documents/BIOPYISICS/step 2/Parameters_solvation/soft/NACCESS/naccess'
srf = NACCESS_atomic(st[0], naccess_binary = NACCESS_BINARY)
io = PDBIO()
st_chains = {}

# Using BioIO trick (see tutorial) to select chains
class SelectChain(Select):
    def __init__(self, chid):
        self.id = chid

    def accept_chain(self, chain):
        if chain.id == self.id:
            return 1
        else:
            return 0

for ch in st[0]:
    io.set_structure(st)
    io.save('tmp.pdb', SelectChain(ch.id))
    if ch.id == "A":
        st_chains[ch.id] = parser.get_structure('stA', 'tmp.pdb')
        add_atom_parameters(st_chains[ch.id], residue_library, ff_params)
        srfA = NACCESS_atomic(st_chains[ch.id][0], naccess_binary=NACCESS_BINARY)
    else:
        st_chains[ch.id] = parser.get_structure('stE', 'tmp.pdb')
        add_atom_parameters(st_chains[ch.id], residue_library, ff_params)
        srfE = NACCESS_atomic(st_chains[ch.id][0], naccess_binary=NACCESS_BINARY)

os.remove('tmp.pdb')


# Interface residues
max_dist = 3.5

if max_dist > 0.:
    interface = get_interface(st, max_dist)


## Initialize Energy aggregates
elec = {}
elec_ala = {}

vdw = {}
vdw_ala = {}

solvAB = {}
solvAB_ala = {}

solvA = {}
solvA_ala = {}

totalIntElec = 0.
totalIntVdw = 0.
totalSolv = 0.
totalSolvMon = {}

## Get the chains ids
chids = []
for ch in st[0]:
    chids.append(ch.id)
    totalSolvMon[ch.id] = 0

total = 0.

for ch in st[0]:
    if ch.id in ["A", "E"]:  # Process only if chain is A or E
        for res in ch.get_residues():
            if max_dist > 0 and res not in interface[ch.id]:
                continue
            elec[res], elec_ala[res], vdw[res], vdw_ala[res] = calc_int_energies(st[0], res)
            solvAB[res], solvAB_ala[res] = calc_solvation(st[0], res)
            solvA[res], solvA_ala[res] = calc_solvation(
                st_chains[ch.id],
                st_chains[ch.id][0][ch.id][res.id[1]]
            )
            totalIntElec += elec[res]
            totalIntVdw += vdw[res]
            totalSolv += solvAB[res]
            totalSolvMon[ch.id] += solvA[res]
            total += elec[res] + vdw[res] + solvAB[res] - solvA[res]

            # Printing energies for each residue in Chain A or E
            print(f"Residue: {res.id[1]}, Chain: {ch.id}")
            print(f"  Electrostatic Int.: {elec[res]}")
            print(f"  Van der Waals Int.: {vdw[res]}")
            print(f"  Solvation Complex: {solvAB[res]}")
            print(f"  Solvation Mono: {solvA[res]}")
            print()

print()
print("Energies when residues in the interface are mutated to alanine")
print('Total Electrostatic Int. =', totalIntElec)
print('Total Vdw Int. =', totalIntVdw)
print('Total Solvation Complex =', totalSolv)
print('Total Solvation', chids[0], '=', totalSolvMon[chids[0]])
print('Total Solvation', chids[1], '=', totalSolvMon[chids[1]])
print('DGint =', total)

Exception: NACCESS did not execute or finish properly.

**Create a single plot for all energy types**

In [34]:
import matplotlib.pyplot as plt

NACCESS_BINARY = '/home/irene/Documents/BIOPYISICS/step 2/Parameters_solvation/soft/NACCESS/naccess'
srf = NACCESS_atomic(st[0], naccess_binary = NACCESS_BINARY)
io = PDBIO()
st_chains = {}

# Using BioIO trick (see tutorial) to select chains
class SelectChain(Select):
    def __init__(self, chid):
        self.id = chid

    def accept_chain(self, chain):
        if chain.id == self.id:
            return 1
        else:
            return 0

for ch in st[0]:
    io.set_structure(st)
    io.save('tmp.pdb', SelectChain(ch.id))
    if ch.id == "A":
        st_chains[ch.id] = parser.get_structure('stA', 'tmp.pdb')
        add_atom_parameters(st_chains[ch.id], residue_library, ff_params)
        srfA = NACCESS_atomic(st_chains[ch.id][0], naccess_binary=NACCESS_BINARY)
    else:
        st_chains[ch.id] = parser.get_structure('stE', 'tmp.pdb')
        add_atom_parameters(st_chains[ch.id], residue_library, ff_params)
        srfE = NACCESS_atomic(st_chains[ch.id][0], naccess_binary=NACCESS_BINARY)

os.remove('tmp.pdb')


# Interface residues
max_dist = 3.5

if max_dist > 0.:
    interface = get_interface(st, max_dist)


## Initialize Energy aggregates
elec = {}
elec_ala = {}

vdw = {}
vdw_ala = {}

solvAB = {}
solvAB_ala = {}

solvA = {}
solvA_ala = {}

totalIntElec = 0.
totalIntVdw = 0.
totalSolv = 0.
totalSolvMon = {}

## Get the chains ids
chids = []
for ch in st[0]:
    chids.append(ch.id)
    totalSolvMon[ch.id] = 0

total = 0.

# Store results for plotting
plot_data = {'Residue': [], 'Electrostatic Int.': [], 'Van der Waals Int.': [], 'Solvation Complex': [], 'Solvation Mono': []}

for ch in st[0]:
    if ch.id in ["A", "E"]:  # Process only if chain is A or E
        for res in ch.get_residues():
            if max_dist > 0 and res not in interface[ch.id]:
                continue
            elec[res], elec_ala[res], vdw[res], vdw_ala[res] = calc_int_energies(st[0], res)
            solvAB[res], solvAB_ala[res] = calc_solvation(st[0], res)
            solvA[res], solvA_ala[res] = calc_solvation(
                st_chains[ch.id],
                st_chains[ch.id][0][ch.id][res.id[1]]
            )
            totalIntElec += elec[res]
            totalIntVdw += vdw[res]
            totalSolv += solvAB[res]
            totalSolvMon[ch.id] += solvA[res]
            total += elec[res] + vdw[res] + solvAB[res] - solvA[res]

            # Add absolute values of data for plotting
            residue_label = f"{res.id[1]}{ch.id}"  # e.g., "45A"
            plot_data['Residue'].append(residue_label)
            plot_data['Electrostatic Int.'].append(abs(elec[res]))
            plot_data['Van der Waals Int.'].append(abs(vdw[res]))
            plot_data['Solvation Complex'].append(abs(solvAB[res]))
            plot_data['Solvation Mono'].append(abs(solvA[res]))

print("Energies when residues in the interface are mutated to alanine")
print('Total Electrostatic Int. =', totalIntElec)
print('Total Vdw Int. =', totalIntVdw)
print('Total Solvation Complex =', totalSolv)
print('Total Solvation', chids[0], '=', totalSolvMon[chids[0]])
print('Total Solvation', chids[1], '=', totalSolvMon[chids[1]])
print('DGint =', total)

# Create a single plot for all energy types
plt.figure(figsize=(12, 6))
plt.title('Absolute Energy Profiles of Residues in Chains A and E')

energy_types = ['Electrostatic Int.', 'Van der Waals Int.', 'Solvation Complex', 'Solvation Mono']
for energy_type in energy_types:
    plt.plot(plot_data['Residue'], plot_data[energy_type], label=energy_type)

plt.xlabel('Residue')
plt.ylabel('Absolute Energy')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Exception: NACCESS did not execute or finish properly.

**Create a plot for DGint**

In [35]:
import matplotlib.pyplot as plt

NACCESS_BINARY = '/home/irene/Documents/BIOPYISICS/step 2/Parameters_solvation/soft/NACCESS/naccess'
srf = NACCESS_atomic(st[0], naccess_binary = NACCESS_BINARY)
io = PDBIO()
st_chains = {}

# Using BioIO trick (see tutorial) to select chains
class SelectChain(Select):
    def __init__(self, chid):
        self.id = chid

    def accept_chain(self, chain):
        if chain.id == self.id:
            return 1
        else:
            return 0

for ch in st[0]:
    io.set_structure(st)
    io.save('tmp.pdb', SelectChain(ch.id))
    if ch.id == "A":
        st_chains[ch.id] = parser.get_structure('stA', 'tmp.pdb')
        add_atom_parameters(st_chains[ch.id], residue_library, ff_params)
        srfA = NACCESS_atomic(st_chains[ch.id][0], naccess_binary=NACCESS_BINARY)
    else:
        st_chains[ch.id] = parser.get_structure('stE', 'tmp.pdb')
        add_atom_parameters(st_chains[ch.id], residue_library, ff_params)
        srfE = NACCESS_atomic(st_chains[ch.id][0], naccess_binary=NACCESS_BINARY)

os.remove('tmp.pdb')


# Interface residues
max_dist = 3.5

if max_dist > 0.:
    interface = get_interface(st, max_dist)


## Initialize Energy aggregates
elec = {}
elec_ala = {}

vdw = {}
vdw_ala = {}

solvAB = {}
solvAB_ala = {}

solvA = {}
solvA_ala = {}

totalIntElec = 0.
totalIntVdw = 0.
totalSolv = 0.
totalSolvMon = {}

## Get the chains ids
chids = []
for ch in st[0]:
    chids.append(ch.id)
    totalSolvMon[ch.id] = 0

total = 0.

# Store results for plotting DGint
plot_data_dgint = {'Residue': [], 'DGint': []}

for ch in st[0]:
    if ch.id in ["A", "E"]:  # Process only if chain is A or E
        for res in ch.get_residues():
            if max_dist > 0 and res not in interface[ch.id]:
                continue
            elec[res], elec_ala[res], vdw[res], vdw_ala[res] = calc_int_energies(st[0], res)
            solvAB[res], solvAB_ala[res] = calc_solvation(st[0], res)
            solvA[res], solvA_ala[res] = calc_solvation(
                st_chains[ch.id],
                st_chains[ch.id][0][ch.id][res.id[1]]
            )
            dgint = elec[res] + vdw[res] + solvAB[res] - solvA[res]
            totalIntElec += elec[res]
            totalIntVdw += vdw[res]
            totalSolv += solvAB[res]
            totalSolvMon[ch.id] += solvA[res]
            total += dgint

            # Add data for plotting DGint
            residue_label = f"{res.id[1]}{ch.id}"  # e.g., "45A"
            plot_data_dgint['Residue'].append(residue_label)
            plot_data_dgint['DGint'].append(abs(dgint))
            
# Create a plot for DGint
plt.figure(figsize=(12, 6))
plt.title('Absolute DGint Energy Profile of Residues in Chains A and E')
plt.plot(plot_data_dgint['Residue'], plot_data_dgint['DGint'], label='DGint')

plt.xlabel('Residue')
plt.ylabel('Absolute DGint Energy')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Exception: NACCESS did not execute or finish properly.

**Plot the DDG values**

In [36]:
import sys
from Bio.PDB import *

NACCESS_BINARY = '/home/irene/Documents/BIOPYISICS/step 2/Parameters_solvation/soft/NACCESS/naccess'
srf = NACCESS_atomic(st[0], naccess_binary = NACCESS_BINARY)
io = PDBIO()
st_chains = {}

# Using BioIO trick (see tutorial) to select chains
class SelectChain(Select):
    def __init__(self, chid):
        self.id = chid

    def accept_chain(self, chain):
        if chain.id == self.id:
            return 1
        else:
            return 0

for ch in st[0]:
    io.set_structure(st)
    io.save('tmp.pdb', SelectChain(ch.id))
    if ch.id == "A":
        st_chains[ch.id] = parser.get_structure('stA', 'tmp.pdb')
        add_atom_parameters(st_chains[ch.id], residue_library, ff_params)
        srfA = NACCESS_atomic(st_chains[ch.id][0], naccess_binary=NACCESS_BINARY)
    else:
        st_chains[ch.id] = parser.get_structure('stE', 'tmp.pdb')
        add_atom_parameters(st_chains[ch.id], residue_library, ff_params)
        srfE = NACCESS_atomic(st_chains[ch.id][0], naccess_binary=NACCESS_BINARY)

os.remove('tmp.pdb')


# Interface residues
max_dist = 3.5

if max_dist > 0.:
    interface = get_interface(st, max_dist)


## Initialize Energy aggregates
elec = {}
elec_ala = {}

vdw = {}
vdw_ala = {}

solvAB = {}
solvAB_ala = {}

solvA = {}
solvA_ala = {}

totalIntElec = 0.
totalIntVdw = 0.
totalSolv = 0.
totalSolvMon = {}

## Get the chains ids
chids = []
for ch in st[0]:
    chids.append(ch.id)
    totalSolvMon[ch.id] = 0

total = 0.

# Initialize lists to store Ala scanning data for plotting
residue_labels = []
ddg_total_values = []

# Perform Ala Scanning and store data for plotting
print("Ala Scanning: DDGs for X->Ala mutations on interface residues")
for ch in st[0]:
    for res in ch.get_residues():
        if max_dist > 0 and res not in interface[ch.id]:
            continue
        ddg_elec = - elec[res] + elec_ala[res]
        ddg_vdw = - vdw[res] + vdw_ala[res]
        ddg_solvAB = - solvAB[res] + solvAB_ala[res]
        ddg_solvA = - solvA[res] + solvA_ala[res]
        ddg_total = ddg_elec + ddg_vdw + ddg_solvAB - ddg_solvA
        
        # Store the residue label and DDG value for plotting
        residue_label = f"{res.id[1]}{ch.id}"  # Construct a residue label, e.g., "45A"
        residue_labels.append(residue_label)
        ddg_total_values.append(ddg_total)

        # Print the DDG values
        print(f"{residue_label} {ddg_elec:11.4f} {ddg_vdw:11.4f} {ddg_solvAB:11.4f} {ddg_solvA:11.4f} {ddg_total:11.4f}")

# Plot the DDG values
plt.figure(figsize=(12, 6))
plt.plot(residue_labels, ddg_total_values, marker='o', linestyle='-')
plt.title('ΔΔG of Ala Scanning on Interface Residues')
plt.xlabel('Residue')
plt.ylabel('ΔΔG (kcal/mol)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Exception: NACCESS did not execute or finish properly.